### Imports

In [1]:
import pickle
import torch
from torch.utils.data import DataLoader, TensorDataset, SequentialSampler
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import csv
import pandas as pd

%run preprocessing.ipynb

c:\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Constants

In [2]:
EMBEDDING_DIM = 300
HIDDEN_SIZE = 512
NUM_LAYERS = 2
BATCH_SIZE = 220
VOCAB_SIZE = len(basic_arabic_letters) + 1
LABELS_SIZE = len(DIACRITICS)

TEST_PATH = "./dataset/test.txt"

with open("./utils/diacritic2id.pickle", 'rb') as file:
    diacritic2id = pickle.load(file)

### Read the model

In [3]:
class CNN(nn.Module):
    def __init__(self, vocab_size=VOCAB_SIZE, num_classes=LABELS_SIZE, embedding_dim=EMBEDDING_DIM, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS):
        super(CNN, self).__init__()
        
        # Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # Convolutional Layer
        self.conv1d = nn.Conv1d(embedding_dim, 256, kernel_size=3, padding=1)
        
        # LSTM Layer
        self.lstm = nn.LSTM(256, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        
        # Linear Layer
        self.linear = nn.Linear(2 * hidden_size, num_classes)

    def forward(self, sentences):
        embeddings = self.embedding(sentences)
        
        # Convolutional Layer
        conv_out = self.conv1d(embeddings.permute(0, 2, 1))
        conv_out = nn.functional.relu(conv_out)
        
        # LSTM Layer
        lstm_out, _ = self.lstm(conv_out.permute(0, 2, 1))
        
        # Linear Layer
        output = self.linear(lstm_out)

        return output

In [4]:
# Instantiate the model
model = CNN()

# Load the saved model weights
model_path = "./models/cnn.pth"
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

print(model)

RuntimeError: Error(s) in loading state_dict for CNN:
	Missing key(s) in state_dict: "lstm.weight_ih_l1", "lstm.weight_hh_l1", "lstm.bias_ih_l1", "lstm.bias_hh_l1", "lstm.weight_ih_l1_reverse", "lstm.weight_hh_l1_reverse", "lstm.bias_ih_l1_reverse", "lstm.bias_hh_l1_reverse". 
	size mismatch for embedding.weight: copying a param with shape torch.Size([37, 200]) from checkpoint, the shape in current model is torch.Size([37, 300]).
	size mismatch for conv1d.weight: copying a param with shape torch.Size([64, 200, 3]) from checkpoint, the shape in current model is torch.Size([256, 300, 3]).
	size mismatch for conv1d.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for lstm.weight_ih_l0: copying a param with shape torch.Size([2048, 64]) from checkpoint, the shape in current model is torch.Size([2048, 256]).
	size mismatch for lstm.weight_ih_l0_reverse: copying a param with shape torch.Size([2048, 64]) from che

### Prepare the testing dataset

In [86]:
test_corpus = readFile(TEST_PATH)

X_test = []
total_len = 0

for sentence in test_corpus:
    char_list, _ = separate_words_and_diacritics(sentence.strip())

    for i in range(len(char_list)):
        X_test.append(char_list[i])
        
x_test_padded = [torch.tensor([char_to_index[char] for char in sentence]) for sentence in X_test]
x_test_padded = pad_sequence(x_test_padded, batch_first=True)

### Testing

In [ ]:
final_predictions = []
total_len = 0
for i in range(len(X_test)):
    sentence = X_test[i]
    sentence = torch.tensor([char_to_index[char] for char in sentence])
    total_len += len(sentence)

    with torch.no_grad():
        predictions = model(sentence.view(sentence.shape[0], 1))
    
    # Get the predictions
    predictions = predictions.argmax(dim=2)
    predictions = [index_to_diacritic[index.item()] for index in predictions]
    predictions = [diacritic2id["" if diacritic == " " else diacritic] for diacritic in predictions]

    final_predictions += predictions

In [97]:
data = []
for i in range(len(final_predictions)):
    data.append({"ID": i, "label": final_predictions[i]})

with open("./test/output.csv", "w", newline="") as csvfile:
    fieldnames = ["ID", "label"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for row in data:
        writer.writerow(row)

### Get the accuracy

In [98]:
# Load your output and gold output CSV files
output_file = "./test/output.csv"
gold_output_file = "./test/test_set_gold.csv"

output_df = pd.read_csv(output_file)
gold_output_df = pd.read_csv(gold_output_file)

# Merge DataFrames on 'ID'
merged_df = pd.merge(output_df, gold_output_df, on="ID", suffixes=('_your', '_gold'))

# Check if the labels match
merged_df['correct'] = merged_df['label_your'] == merged_df['label_gold']

# Calculate accuracy
accuracy = merged_df['correct'].mean()

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 29.81%
